In [1]:
import time
import math
import copy
import torch
import pickle
import random
import logging
import warnings
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import torch.nn as nn
import torch.optim as opt
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from torch.autograd import grad
from torch.autograd.functional import vhp
from get_datasets import get_diabetes, get_adult, get_law
from torch.utils.data import Subset, DataLoader
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score
from scipy.stats import spearmanr

plt.rcParams['figure.dpi'] = 300
warnings.filterwarnings("ignore")

E = math.e

### Utility Functions

In [2]:
def visualize_result(e_k_actual, e_k_estimated, ep, k_):

    #e_k_estimated = [-1*ek for ek in e_k_estimated]
    fig, ax = plt.subplots()
    palette = sns.color_palette("cool", len(e_k_actual))
    sns.set(font_scale=1.15)
    sns.set_style(style='white')
    min_x = np.min(e_k_actual)
    max_x = np.max(e_k_actual)
    min_y = np.min(e_k_estimated)
    max_y = np.max(e_k_estimated)
    
    plt.rcParams['figure.figsize'] = 6, 5
    z = np.polyfit(e_k_actual,  e_k_estimated, 1)
    p = np.poly1d(z)
    xx = np.linspace(-p(2)/p(1), max(e_k_actual)+.0001)
    yy = np.polyval(p, xx)
    #add trendline to plot
    ax.plot(xx, yy, ls="-", color='k')
    for k in range(len(e_k_actual)):
        ax.scatter(e_k_actual[k], e_k_estimated[k], zorder=2, s=45, color = palette[k], label=ep[k])

    ax.set_title(f'Actual vs. Estimated loss for k={k_:.2f}%')
    ax.set_xlabel('Actual loss difference')
    ax.set_ylabel('Estimated loss difference')
   
    ax.set_xlim(min_x-.0001, max_x+.0001)
    ax.set_ylim(min_y-.0001, max_y+.0001)
    
   
    text = 'MAE = {:.03}\nP = {:.03}'.format(mean_absolute_error(e_k_actual, e_k_estimated), spearmanr(e_k_actual, e_k_estimated).correlation)
    #ax.text(max_x+.00009,min_y-.00008, text, verticalalignment='bottom', horizontalalignment='right')
    #ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #plt.tight_layout()
    plt.xticks(rotation = 45)
    plt.show()
    # cooler color = smaller epsilon

In [3]:
 class CreateData(torch.utils.data.Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.targets[idx]

        return out_data, out_label

In [4]:
def get_data(new_train_df, feature_set, label, k):    

    selected_group = new_train_df.loc[new_train_df['sex'] == 0]

    num_to_sample = round((k / 100)*len(selected_group))

    sampled_group = selected_group.sample(n=num_to_sample)
    not_selected = new_train_df.drop(sampled_group.index)

    selected_group_X = sampled_group[feature_set]
    selected_group_y = sampled_group[label]

    not_selected_group_X = not_selected[feature_set]
    not_selected_group_y = not_selected[label]   
    
    return selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y

### Randomized Response

In [5]:
def get_p(epsilon):
    probability = float(E ** epsilon) / float(1 + (E ** epsilon))
    p = torch.FloatTensor([[probability, 1-probability], [1-probability, probability]])
    
    return p

### Models

In [6]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, num_features):
        super(LogisticRegression, self).__init__()
        
        self.fc1 = torch.nn.Linear(num_features, 1, bias=False)
        self.criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
        
    def forward(self, x):
        logits = self.fc1(x)

        return logits
    
    def loss(self, logits, y):
        loss = self.criterion(logits.ravel(), y)
        
        probabilities = torch.sigmoid(logits)
        thresh_results = []
        
        for p in probabilities:
            if p>.5:
                thresh_results.append(1)
            else:
                thresh_results.append(0)
                
        num_correct = 0
        for r,y_ in zip(thresh_results, y):
            if r == y_:
                num_correct += 1
                
        acc = num_correct / len(y)
        
        return loss, acc

In [7]:
def train(model, dataset):
    model.train()
    
    opt = torch.optim.SGD(model.parameters(), lr=.005, weight_decay=0)
    criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')

            
    train_data = CreateData(dataset[0], dataset[1])
    train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)

    for itr in range(0, 7):
        for i, [x,y] in enumerate(train_dataloader):
            opt.zero_grad()
            oupt = model(x)
            
            try:
                loss_val = criterion(oupt.ravel(), y)
            except ValueError:
                loss_val = criterion(oupt, y)
            
            loss_val.backward()
            opt.step() 
            
    return model

### Influence Calculation Functions


In [8]:
def calc_influence_single(model, epsilon, train_data, test_data, group_data, device, num_features, criterion):
    start = time.time()
    est_hess = explicit_hess(model, train_data, device, criterion)

    grad_test = grad_z([test_data[0], test_data[1]], model, device, criterion)
    s_test_vec = torch.mm(grad_test[0], est_hess.to(device))

    P = get_p(epsilon)
    
    p_01, p_10 = P[0][1].item(), P[1][0].item()
    
    pi_1 = sum(list(group_data[1]))
    pi_0 = len(group_data[1]) - pi_1
    
    lam_0 = round(p_01 * pi_1)
    lam_1 = round(p_10 * pi_0)

    S_pert = 1 - group_data[1]
    
    y_w_group_pert = pd.concat([group_data[3], S_pert], axis = 0, ignore_index=True)
    y_wo_pert = pd.concat([group_data[3], group_data[1]], axis = 0, ignore_index=True)
    reconstructed_x = pd.concat([group_data[2], group_data[0]], axis = 0, ignore_index=True)
  
    assert len(S_pert) == len(group_data[1])
    grad_z_vec = grad_training([group_data[0],group_data[1]], S_pert, [model], device, [lam_0, lam_1, epsilon], criterion)
    
    influence = torch.dot(s_test_vec.flatten(), grad_z_vec[0].flatten()) * (1/len(train_data[0]))
    #print(influence)
    end = time.time() - start

    return influence.cpu(), end

In [9]:
def explicit_hess(model, train_data, device, criterion):
 
    logits = model(train_data[0])
    loss = criterion(logits.ravel(), train_data[1]) #reduction mean
    
    grads = grad(loss, model.parameters(), retain_graph=True, create_graph=True)

    hess_params = torch.zeros(len(model.fc1.weight[0]), len(model.fc1.weight[0]))
    for i in range(len(model.fc1.weight[0])):
        hess_params_ = grad(grads[0][0][i], model.parameters(), retain_graph=True)[0][0]
        for j, hp in enumerate(hess_params_):
            hess_params[i,j] = hp
    
    inv_hess = torch.linalg.inv(hess_params)
    return inv_hess

In [10]:
def grad_z(test_data, model, device, criterion):

    model.eval()

    test_data_features = test_data[0]
    test_data_labels = test_data[1]

    logits = model(test_data_features)
    loss = criterion(logits, torch.atleast_2d(test_data_labels).T) # reduction mean
    
    return grad(loss, model.parameters())

In [11]:
def grad_training(train_data, y_perts, parameters, device, epsilon, criterion):
    
    criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')
    
    lam_0, lam_1, ep = epsilon
    lam = lam_0 + lam_1
    len_s = len(y_perts)
    
    train_data_features = torch.FloatTensor(train_data[0].values).to(device)
    train_data_labels = torch.FloatTensor(train_data[1].values).to(device)
    train_pert_data_labels = torch.FloatTensor(y_perts.values).to(device)
    
    model = parameters[0]
    model.eval()

    logits = model(train_data_features)

    orig_loss = criterion(logits, torch.atleast_2d(train_data_labels).T)
    pert_loss = criterion(logits, torch.atleast_2d(train_pert_data_labels).T)
    loss = float(1/(1 + (E ** ep)))*(pert_loss - orig_loss)
    
    to_return = grad(loss, model.parameters())
    
        
    return to_return

### Main Function

In [12]:
def Main(dataset, epsilons, ks, num_rounds):

    device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
    criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
    
    all_orig_loss_e_k = []
    all_est_loss_e_k = []
    all_time = []
    
    for nr in range(num_rounds):
        print(f'\nRound {nr+1}')
        ############
        # Get data #
        ############
        print('\nGetting Data...')
        if dataset == 'adult':
            data = get_adult()
            label = 'income_class'
        elif dataset == 'diabetes':
            data = get_diabetes()
            label = 'readmitted'
        else:
            data = get_law()
            label = 'admit'

        feature_set = set(data.columns) - {label}
        num_features = len(feature_set)
    
        X = data[feature_set]
        y = data[label]

        if dataset == 'diabetes':
            undersample = RandomUnderSampler(random_state=42)
            new_X, new_y = undersample.fit_resample(X, y)
        else:
            new_X = X
            new_y = y

        X_train, X_test, y_train, y_test = train_test_split(new_X, new_y, test_size=0.20, random_state=42)
  
        new_train_df = pd.concat([X_train, y_train], axis=1)
  
        train_sample_num = len(X_train)
    
        x_test_input = torch.FloatTensor(X_test.values).to(device)
        y_test_input = torch.FloatTensor(y_test.values).to(device)

        x_train_input = torch.FloatTensor(X_train.values).to(device)
        y_train_input = torch.FloatTensor(y_train.values).to(device)
   
        ##############################################
        # Train original model and get original loss #
        ##############################################
        print('Training original model...')
        start = time.time()
        torch_model = LogisticRegression(num_features)
        torch.save(torch_model.state_dict(), 'models/initial_config.pth')
        torch_model.to(device)
        torch_model = train(torch_model, [x_train_input, y_train_input])
        end = time.time() - start
        
        print(f'Total model training time: {end:.3f}')
        test_loss_ori, acc_ori = torch_model.loss(torch_model(x_test_input), y_test_input)
        
        e_k_act_losses = []
        e_k_est_losses = []
        influence_time = []
        
        ################################################################
        # Perform influence and retraining for all epsilons a k values #
        ################################################################
        print('\nBegining epsilon and k rounds')
        print('-----------------------------')
        for k in ks:
            print(f'\nk: {k}')
            
            k_act_losses = []
            k_est_losses = []
            inf_time = []
            total_time = 0
            for ep in epsilons:
                # Influence
                print(f'ep: {ep}')
                selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y = get_data(new_train_df, feature_set, label, k)

#                 loss_diff_approx, tot_time = calc_influence_single(torch_model, ep, [x_train_input, y_train_input], [x_test_input, y_test_input], [selected_group_X, selected_group_y, not_selected_group_X, not_selected_group_y], device, num_features, criterion)
#                 total_time += tot_time
#                 loss_diff_approx = -torch.FloatTensor(loss_diff_approx).cpu().numpy()
                #print(f'Ep {ep} retrain time: {tot_time:.3f}')
                # Retrain
                start = time.time()
                P = get_p(ep)

                p_01, p_10 = P[0][1].item(), P[1][0].item()

                pi_1 = sum(list(selected_group_y))
                pi_0 = len(selected_group_y) - pi_1

                lam_0 = round(p_01 * pi_1)
                lam_1 = round(p_10 * pi_0)

                S = pd.concat([selected_group_X, selected_group_y], axis=1, ignore_index=False)

                G0 = S[label][S[label].eq(1)].sample(lam_0).index
                G1 = S[label][S[label].eq(0)].sample(lam_1).index

                G = S.loc[G0.union(G1)]
                not_g = S.drop(G0.union(G1))

                G_pert = 1 - G[label]

                y_w_group_pert = pd.concat([not_selected_group_y, not_g[label], G_pert], axis = 0, ignore_index=True)
                y_wo_pert = pd.concat([not_selected_group_y, not_g[label], G[label]], axis = 0, ignore_index=True)
                reconstructed_x = pd.concat([not_selected_group_X, not_g[feature_set], G[feature_set]], axis = 0, ignore_index=True)

                model_pert = LogisticRegression(num_features)
                model_pert.load_state_dict(torch.load('models/initial_config.pth'))
                model_pert.to(device)
                model_pert = train(model_pert, [torch.FloatTensor(reconstructed_x.values).to(device), torch.FloatTensor(y_w_group_pert.values).to(device)])
                test_loss_retrain, acc_retrain = model_pert.loss(model_pert(x_test_input), y_test_input)

                 # get true loss diff
                loss_diff_true = (test_loss_retrain - test_loss_ori).detach().cpu().item()
                print(f'Retrain time: {time.time() - start:.3f}')
#                 print(loss_diff_true)
#                 k_act_losses.append(loss_diff_true)
#                 k_est_losses.append(loss_diff_approx)
#                 inf_time.append(tot_time)
            print(f'Total influence time: {total_time}, Average influence time: {total_time/len(epsilons)}')
#             e_k_act_losses.append(k_act_losses)
#             e_k_est_losses.append(k_est_losses)
#             influence_time.append(inf_time)
            
#         all_orig_loss_e_k.append(e_k_act_losses)
#         all_est_loss_e_k.append(e_k_est_losses) 
#         all_time.append(influence_time)
    
#     return all_orig_loss_e_k, all_est_loss_e_k, all_time

### Perform Experiment 

#### Constants

In [13]:
epsilons = [.01, .02, .03, .04, .05, .06, .07, .08, .09, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
k = np.linspace(1, 25, 10)
rounds = 5

In [14]:
print(k)

[ 1.          3.66666667  6.33333333  9.         11.66666667 14.33333333
 17.         19.66666667 22.33333333 25.        ]


In [15]:
all_orig_loss_e_k, all_est_loss_e_k, all_time = Main('adult', epsilons, k, rounds)
# [round1[epsilon1[k1,...k10], epsilon2[k1,...k10],...], round2[...]]     

with open('all_orig_loss_e_k.txt', "wb") as file:   #Pickling
    pickle.dump(all_orig_loss_e_k, file)

with open('all_est_loss_e_k.txt', "wb") as file2:   #Pickling
    pickle.dump(all_est_loss_e_k, file2)


Round 1

Getting Data...
Training original model...
Total model training time: 3.403

Begining epsilon and k rounds
-----------------------------

k: 1.0
ep: 0.01
Retrain time: 3.060
ep: 0.02
Retrain time: 3.038
ep: 0.03
Retrain time: 3.019
ep: 0.04
Retrain time: 3.302
ep: 0.05
Retrain time: 3.313
ep: 0.06
Retrain time: 3.247
ep: 0.07
Retrain time: 3.149
ep: 0.08
Retrain time: 3.201
ep: 0.09
Retrain time: 3.332
ep: 0.1
Retrain time: 3.405
ep: 0.2
Retrain time: 2.803
ep: 0.3
Retrain time: 3.166
ep: 0.4
Retrain time: 3.025
ep: 0.5
Retrain time: 3.474
ep: 0.6
Retrain time: 2.987
ep: 0.7
Retrain time: 3.359
ep: 0.8
Retrain time: 3.240
ep: 0.9
Retrain time: 2.898
ep: 1
Retrain time: 3.036
ep: 2
Retrain time: 3.089
ep: 3
Retrain time: 3.094
ep: 4
Retrain time: 3.145
ep: 5
Retrain time: 3.325
ep: 6
Retrain time: 3.179
ep: 7
Retrain time: 3.252
ep: 8
Retrain time: 2.996
ep: 9
Retrain time: 3.094
ep: 10
Retrain time: 3.250
Total influence time: 0, Average influence time: 0.0

k: 3.666666666666

Retrain time: 3.384
ep: 0.8
Retrain time: 3.081
ep: 0.9
Retrain time: 3.256
ep: 1
Retrain time: 2.932
ep: 2
Retrain time: 3.679
ep: 3
Retrain time: 3.525
ep: 4
Retrain time: 3.149
ep: 5
Retrain time: 3.311
ep: 6
Retrain time: 3.054
ep: 7
Retrain time: 3.505
ep: 8
Retrain time: 2.891
ep: 9
Retrain time: 3.587
ep: 10
Retrain time: 3.130
Total influence time: 0, Average influence time: 0.0

Round 2

Getting Data...
Training original model...
Total model training time: 2.780

Begining epsilon and k rounds
-----------------------------

k: 1.0
ep: 0.01
Retrain time: 3.361
ep: 0.02
Retrain time: 3.146
ep: 0.03
Retrain time: 3.019
ep: 0.04
Retrain time: 3.299
ep: 0.05
Retrain time: 3.614
ep: 0.06
Retrain time: 4.085
ep: 0.07
Retrain time: 3.703
ep: 0.08
Retrain time: 3.263
ep: 0.09
Retrain time: 3.620
ep: 0.1
Retrain time: 3.484
ep: 0.2
Retrain time: 3.720
ep: 0.3
Retrain time: 3.562
ep: 0.4
Retrain time: 3.636
ep: 0.5
Retrain time: 3.324
ep: 0.6
Retrain time: 3.507
ep: 0.7
Retrain time: 3.45

KeyboardInterrupt: 

In [ ]:
all_time

In [ ]:
with open('all_orig_loss_e_k.txt', 'rb') as f:
    all_orig_loss_e_k = pickle.load(f)
    
with open('all_est_loss_e_k.txt', 'rb') as f:
    all_est_loss_e_k = pickle.load(f)

In [ ]:
# [actual, estimate]

sum_orig_loss_e_k = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]
sum_est_loss_e_k = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]
sum_time = [[0 for _ in range(len(k))] for _ in range(len(epsilons))]

avg_orig_loss = []
avg_est_loss = []
avg_time = []

for round_ in range(len(all_orig_loss_e_k)):
    for e in range(len(epsilons)):
        for k_ in range(len(k)):
            sum_orig_loss_e_k[e][k_] = sum_orig_loss_e_k[e][k_] + all_orig_loss_e_k[round_][e][k_]
            sum_est_loss_e_k[e][k_] = sum_est_loss_e_k[e][k_] + all_est_loss_e_k[round_][e][k_]
            sum_time[e][k_] = sum_time[e][k_] + all_time[round_][e][k_]

for e in range(len(epsilons)):
    avg_orig_loss.append([ elem / len(all_orig_loss_e_k) for elem in sum_orig_loss_e_k[e]])
    avg_est_loss.append([elem/ len(all_orig_loss_e_k) for elem in sum_est_loss_e_k[e]])
    avg_time.append([elem/ len(all_orig_loss_e_k) for elem in sum_time[e]])

k_e_orig = [[] for _ in range(len(k))]
k_e_est = [[] for _ in range(len(k))]

for e in range(len(epsilons)):
    for k_ in range(len(k)):
        k_e_orig[k_].append(avg_orig_loss[e][k_])
        k_e_est[k_].append(avg_est_loss[e][k_])
        
print(k_e_est)

averaged_time = []

for e in range(len(epsilons)):
    averaged_time.append(sum_time[e][0])

average_time_final = sum(averaged_time) / len(averaged_time)

In [ ]:
k = np.linspace(1, 25, 10)
k_ = list(k)
k_

In [ ]:
for i in range(len(k_e_orig)):
    visualize_result(k_e_orig[i], k_e_est[i], epsilons, k_[i])

In [ ]:
spearman_g_ep = []
spearman_g_k = []

mae_g_ep = []
mae_g_k = []
    
for i in range(len(avg_orig_loss)):
    

    spearman = []
    mae = []
    
    for j in range(len(actual)):
        spearman.append(spearmanr(actual[j], estimated[j]).correlation)
        mae.append(mean_absolute_error(actual[j], estimated[j]))
        
    spearman_g_ep.append(spearman)
    mae_g_ep.append(mae)
    
for i, kge in enumerate(k_g_e):
    actual = [[x[0] for x in kge[0]], [x[0] for x in kge[1]]]#, [x[0] for x in kge[2]], [x[0] for x in kge[3]], [x[0] for x in kge[4]]]
    estimated = [[x[1] for x in kge[0]], [x[1] for x in kge[1]]]#, [x[1] for x in kge[2]], [x[1] for x in kge[3]], [x[1] for x in kge[4]]]

    spearman = []
    mae = []
    
    for j in range(len(actual)):
        spearman.append(spearmanr(actual[j], estimated[j]).correlation)
        mae.append(mean_absolute_error(actual[j], estimated[j]))
        
    spearman_g_k.append(spearman)
    mae_g_k.append(mae)


In [ ]:
print(avg_orig_loss[4])
print(avg_est_loss[4])
print()
print(avg_orig_loss[13])
print(avg_est_loss[13])
print()
print(avg_orig_loss[22])
print(avg_est_loss[22])
print()
print(avg_orig_loss[ -5])
print(avg_est_loss[-5])

In [ ]:
len(spearman_g_ep), len(spearman_g_ep[0])
spearman_g_ep

In [ ]:
len(spearman_g_k), len(spearman_g_k[0])
spearman_g_k

In [ ]:
len(mae_g_ep), len(mae_g_ep[0])
mae_g_ep

In [ ]:
len(mae_g_k), len(mae_g_k[0])
mae_g_k

In [ ]:
rank_est = [22, 20,23,21,19,18,16,17,15,14,12,13,11,10,9,8,7,6,4,5,3,2,1]
rank_act = [23,20,21,22,17,18,16,19,14,15,13,12,11,10,8,7,9,6,4,5,2,3,1]
a= np.cov(np.array(rank_est), np.array(rank_act), bias=True)[0][1]

In [ ]:
b = np.std(np.array(rank_est))

In [ ]:
c = np.std(np.array(rank_act))

In [ ]:
a /(b*c)

In [ ]:
print(a, b, c)